In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, regexp_extract

In [2]:
spark = pyspark.sql.SparkSession.builder.appName('practice').getOrCreate()

In [3]:
# 1 - Lendo arquivo 'video-stats'

video_stats_df = spark.read.csv('/content/drive/MyDrive/Material de apoio - M27 Projeto/videos-stats.csv', header=True, inferSchema=True)
video_stats_df.show(5)

+---+--------------------+-----------+------------+-------+-------+--------+---------+
|_c0|               Title|   Video ID|Published At|Keyword|  Likes|Comments|    Views|
+---+--------------------+-----------+------------+-------+-------+--------+---------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech|76779.0|  4306.0|1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech|63825.0|  3338.0|1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech|71566.0|  1426.0| 922918.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech|96513.0|  5155.0|1855644.0|
+---+--------------------+-----------+------------+-------+-------+--------+---------+
only showing top 5 rows



In [33]:
# 2 - Alterando valores nulos para 0

video_likes_null = video_stats_df.fillna({'Likes': 0})
video_comments_null = video_stats_df.fillna({'Comments': 0})
video_views_null = video_stats_df.fillna({'Views': 0})

video_likes_null.filter(col('Likes') == 0).count()
video_comments_null.filter(col('Comments') == 0).count()
video_views_null.filter(col('Views') == 0).count()


Number of null values in Likes column after filling: 0


In [4]:
# 3 - Lendo arquivo 'comments'

comments_df = spark.read.csv('/content/drive/MyDrive/Material de apoio - M27 Projeto/comments.csv', header=True, inferSchema=True)
comments_df.show(5)

+---+-----------+--------------------+-----+---------+
|_c0|   Video ID|             Comment|Likes|Sentiment|
+---+-----------+--------------------+-----+---------+
|  0|wAZZ-UWGVHI|Let's not forget ...| 95.0|      1.0|
|  1|wAZZ-UWGVHI|Here in NZ 50% of...| 19.0|      0.0|
|  2|wAZZ-UWGVHI|I will forever ac...|161.0|      2.0|
|  3|wAZZ-UWGVHI|Whenever I go to ...|  8.0|      0.0|
|  4|wAZZ-UWGVHI|Apple Pay is so c...| 34.0|      2.0|
+---+-----------+--------------------+-----+---------+
only showing top 5 rows



In [5]:
# 4 - Calcular a quantidade total de registros

print('Total Registros DF Video: ', video_stats_df.count(), f'\n Total de Registros DF Comments', comments_df.count(),f'\n Total Registros: ', video_stats_df.count() + comments_df.count())


Total Registros DF Video:  1881 
 Total de Registros DF Comments 30036 
 Total Registros:  31917


In [29]:
# 5 - Removendo ID nulos da coluna Video ID nas duas tabelas

video_null_video_id = video_stats_df.filter(col('Video ID').isNull())
remove_video_null = video_null_video_id.na.drop(subset=['Video ID'])
print(remove_video_null.count())

comments_null_video_id = video_stats_df.filter(col('Video ID').isNull())
remove_comments_null = comments_null_video_id.na.drop(subset=['Video ID'])
print(remove_comments_null.count())



0
0


In [16]:
# 6 - Removendo duplicatas

remove_dupli_video_stats = video_stats_df.dropDuplicates(['Video ID'])
remove_dupli_video_stats.count()


1869

In [139]:
# 7 - Casting de colunas

def muda_tipo_coluna(df, colunas, tipo_coluna):
  for coluna in colunas:
    df = df.withColumn(coluna, df[coluna].cast(tipo_coluna))
  return df.printSchema()

muda_tipo_coluna(video_stats_df, ['Likes', 'Comments', 'Views'], 'int')

root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)



In [162]:
# 8 Muda nome e casting

muda_tipo_coluna(comments_df, ['Likes', 'Sentiment'], 'int')
comments_df.withColumnRenamed('Likes', 'Likes Comment').show(5)


root
 |-- _c0: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Sentiment: integer (nullable = true)

+---+-----------+--------------------+-------------+---------+
|_c0|   Video ID|             Comment|Likes Comment|Sentiment|
+---+-----------+--------------------+-------------+---------+
|  0|wAZZ-UWGVHI|Let's not forget ...|         95.0|      1.0|
|  1|wAZZ-UWGVHI|Here in NZ 50% of...|         19.0|      0.0|
|  2|wAZZ-UWGVHI|I will forever ac...|        161.0|      2.0|
|  3|wAZZ-UWGVHI|Whenever I go to ...|          8.0|      0.0|
|  4|wAZZ-UWGVHI|Apple Pay is so c...|         34.0|      2.0|
+---+-----------+--------------------+-------------+---------+
only showing top 5 rows



In [63]:
# 9 - Criando campos com soma

interaction_sum = video_stats_df.withColumn('interaction_sum', col('Likes') + col('Comments') + col('Views'))
interaction_sum.show(5)


+---+--------------------+-----------+------------+-------+-------+--------+---------+---------------+
|_c0|               Title|   Video ID|Published At|Keyword|  Likes|Comments|    Views|interaction_sum|
+---+--------------------+-----------+------------+-------+-------+--------+---------+---------------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|       139691.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech|76779.0|  4306.0|1758063.0|      1839148.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech|63825.0|  3338.0|1564007.0|      1631170.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech|71566.0|  1426.0| 922918.0|       995910.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech|96513.0|  5155.0|1855644.0|      1957312.0|
+---+--------------------+-----------+------------+-------+-------+--------+---------+---------------+
only showing top 5 rows



In [163]:
# 10 Conversão de datas
from pyspark.sql.functions import to_date, date_format

br_date = video_stats_df.withColumn('data', to_date(col('Published At')))


br_date.printSchema()


root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: double (nullable = true)
 |-- Comments: double (nullable = true)
 |-- Views: double (nullable = true)
 |-- data: date (nullable = true)



In [89]:
# 11 Extraindo apenas o ano do dataframe

pub_ano = br_date.select(date_format(col('data'), 'yyyy'))

pub_ano.show(5)

+-----------------------+
|date_format(data, yyyy)|
+-----------------------+
|                   2022|
|                   2022|
|                   2022|
|                   2022|
|                   2022|
+-----------------------+
only showing top 5 rows



In [192]:
# 12 Fazendo join das tabelas

df_join_video_comments = video_stats_df.alias('df1').join(comments_df.alias('df2'), 'Video ID', 'inner').\
select(col('df1.Video ID'),
    col('df1._c0').alias('_c0'),
    col('df1.Title'),
    col('df1.Published At'),
    col('df1.Keyword'),
    col('df1.Likes'),
    col('df1.Comments'),
    col('df1.Views'),
    col('df2.Comment'))


df_join_video_comments.show(5)

df_join_video_comments.printSchema()

+-----------+---+--------------------+------------+-------+------+--------+--------+--------------------+
|   Video ID|_c0|               Title|Published At|Keyword| Likes|Comments|   Views|             Comment|
+-----------+---+--------------------+------------+-------+------+--------+--------+--------------------+
|wAZZ-UWGVHI|  0|Apple Pay Is Kill...|  2022-08-23|   tech|3407.0|   672.0|135612.0|Let's not forget ...|
|wAZZ-UWGVHI|  0|Apple Pay Is Kill...|  2022-08-23|   tech|3407.0|   672.0|135612.0|Here in NZ 50% of...|
|wAZZ-UWGVHI|  0|Apple Pay Is Kill...|  2022-08-23|   tech|3407.0|   672.0|135612.0|I will forever ac...|
|wAZZ-UWGVHI|  0|Apple Pay Is Kill...|  2022-08-23|   tech|3407.0|   672.0|135612.0|Whenever I go to ...|
|wAZZ-UWGVHI|  0|Apple Pay Is Kill...|  2022-08-23|   tech|3407.0|   672.0|135612.0|Apple Pay is so c...|
+-----------+---+--------------------+------------+-------+------+--------+--------+--------------------+
only showing top 5 rows

root
 |-- Video ID: s

In [92]:
# 13 Lendo o terceiro arquivo

us_video_df = spark.read.csv('/content/drive/MyDrive/Material de apoio - M27 Projeto/USvideos.csv', header=True, inferSchema=True)
us_video_df.show(5)

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [99]:
# 14  Fazendo tratamento de titulos e join de tabelas

trat_video = us_video_df.withColumnRenamed('tile', 'Title')
trat_video.printSchema()

df_join_video_usvideo = video_stats_df.join(video_stats_df, 'Title').join(trat_video, 'Title')
df_join_video_usvideo.show(5)

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)

+------------------+----+-----------+------------+----------------+-------+--------+--------+----+-----------+------------+----------------+-------+--------+--------+-----------+-------------+-----------------+-----------+--------------------+--------------------+-------+------+--------+-------------+---

In [101]:
# 15 Contando todos valores nulos por coluna

from pyspark.sql.functions import col, sum

null_counts = video_stats_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in video_stats_df.columns])
null_counts.show()

+---+-----+--------+------------+-------+-----+--------+-----+
|_c0|Title|Video ID|Published At|Keyword|Likes|Comments|Views|
+---+-----+--------+------------+-------+-----+--------+-----+
|  0|    0|       0|           0|      0|    2|       2|    2|
+---+-----+--------+------------+-------+-----+--------+-----+



In [195]:
# 16 Criando funções para exclusão de colunas e para salvar o DF

def remove_col(df, colunas):
  for coluna in colunas:
    df = df.drop(coluna)

  if coluna not in df.columns:
    print('Colunas removidas com sucesso!')

  return df

def salva_df(df, formato, local):
  writer = df.write.mode('overwrite').option('header', 'true')
  if formato == 'csv':
    writer.csv(local)
  elif formato == 'parquet':
    writer.parquet(local)
  elif formato == 'json':
    writer.json(local)
  else:
    print(f'Formato {formato} não suportado')

    return df

remove_col(video_stats_df, ['_c0'])
salva_df(video_stats_df, 'parquet', '/content/drive/MyDrive/Material de apoio - M27 Projeto/video-tratados-parquet')

Colunas removidas com sucesso!


In [190]:
# 17 Executando as funções criadas no passo anterior

remove_col(video_stats_df, ['_c0'])
salva_df(video_stats_df, 'parquet', '/content/drive/MyDrive/Material de apoio - M27 Projeto/videos-comments-tratados-parquet')
